Fazer todos esses pré processamentos para o conjunto de dados é desgastante. Há sim uma maneira de 'automatizar' isso, essa maneria constrói como que um canal por onde passam os nossos dados. O que chamamos de **pipeline**.

Antes de vermos o que é isso, vamos novamente fazer os imports, ler e limpar os nossos dados.

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [0]:
dados_treino = pd.read_csv('treino.csv', index_col='Id')
dados_teste = pd.read_csv('teste.csv', index_col='Id')

Fazendo cópia, removendo linhas sem o valor que queremos prever e a coluna que vamos prever dos dados de treino

In [0]:
X = dados_treino.copy()

In [0]:
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

Feito essa limpeza, dividimos os datasets entre treino e teste.

In [0]:
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, 
                                                        train_size=0.8, 
                                                        test_size=0.2, 
                                                        random_state=42)

Dos dados de treino, vamos selecionar as colunas categóricas com baixa cardinalidade, menor do que 10.

In [0]:
colunas_cat = [coluna for coluna in X_treino.columns 
               if X_treino[coluna].nunique() < 10 
               and X_treino[coluna].dtype == "object"]

Agora vamos selecionar as colunas numéricas.

In [0]:
colunas_num = [coluna for coluna in X_treino.columns 
               if X_treino[coluna].dtype in ['int64', 'float64']]

Das colunas que selecionamos, vamos manter apenas elas nos dados de validacao, treino e teste.

Como fazemos para usar todas as colunas que selecionamos? Juntamos com +.

In [0]:
colunas_selecionadas = colunas_cat + colunas_num

Feito isso, vamos usar apenas essas colunas nos nossos dados de treino, teste e validação. Então vamos copia-las para novos Xs.

In [0]:
X_treino_selec = X_treino[colunas_selecionadas].copy()
X_valid_selec = X_valid[colunas_selecionadas].copy()
X_teste_selec = dados_teste[colunas_selecionadas].copy()

Beleza, agora podemos organizar a nossa pipeline. A pipeline é uma sequência de passos. Esses passos geralmente involvem as etapas de preprocessamento e um modelo. Chamamos esses passos de steps.

Ela vai ser algo assim:

In [10]:
steps = [(preprocessador, modelo)]

pipeline_1 = Pipeline(steps)

NameError: ignored

Para isso, precisamos criar os preprocessadores. Quais são eles?

Os imputadores e encoders para processar os dados numericos e categoricos. Certo?

Como eles usam o fit transform, vamos chama-los de transformers.

Primeiro, para transformar os dados numéricos, vamos usar o simple imputer. 

In [0]:
transformer_num = SimpleImputer(strategy='mean')

E para os nossos dados categóricos, queremos usar o simple imputer e o one hot encoding, certo? Vamos escrevê-los.

In [0]:
transformer_cat = (SimpleImputer(strategy='constant'),
                   OneHotEncoder(handle_unknown='ignore'))

E podemos especificar o nome de cada um desses processos.

In [0]:
transformer_cat = ('simple imputer', SimpleImputer(strategy='constant')),('one hot encoder', OneHotEncoder(handle_unknown='ignore'))

Podemos também colocar essas tuplas em uma lista.

In [0]:
transformer_cat = [('simple imputer', SimpleImputer(strategy='constant')),
                   ('one hot encoder', OneHotEncoder(handle_unknown='ignore'))]

Como temos o passo 1 que é usar o simple imputer e o passo 2 que é usar o one hot ecoding, vou chamar a nossa lista de steps.

Isso lembra alguma coisa? Sim aquela pipeline que tinha os steps.

In [0]:
steps=[('imputer', SimpleImputer(strategy='constant')),
       ('onehot', OneHotEncoder(handle_unknown='ignore'))]

Repare que aqui nas variáveis categóricas já temos duas coisas acontecendo, dois passos, portanto, podemos usar uma pipeline.

In [0]:
transformer_cat = Pipeline(steps)

Agora vamos pegar cada um desses transformers e também colocá-los em uma lista.

In [0]:
transformers=[('transformer_num', transformer_num, colunas_num),
              ('transformer_cat', transformer_cat, colunas_cat)]

E vamos definir o nosso preprocessador, ele faz o que? Pre processa as nossas colunas, certo? Então vai ser uma classe do tipo ColumnTransformer que recebe os nossos transformers.

In [0]:
preprocessador = ColumnTransformer(transformers)

Se olharmos no exemplo da pipeline inicial, temos os preprocessadores e o modelo, certo? Então vamos definir o modelo.

In [0]:
modelo = RandomForestRegressor(n_estimators=50, random_state=42)

Com modelo e preprocessador definidos, basta reunirmos ambos em steps e colocá-los dentro de uma pipeline.

In [0]:
pipeline_1 = Pipeline(steps=[('preprocessador', preprocessador),
                             ('modelo', modelo)])

Repare que é melhor declarar os steps dentro do objeto, uma vez que ele fica atrelado à uma pipeline específica.

Pronto, agora temos a nossa pipeline pronta e sempre que tivermos algum dado, ao invés de fazermos manualmente como até agora, ele irá passar pelos imputers e one hot encoding automaticamente. 

Assim como para um modelo, temos os métodos fit e predict, veja que não precisamos mais 'arrumar' os dados agora que temos a pipeline.

In [21]:
pipeline_1.fit(X_treino_selec, y_treino)

Pipeline(memory=None,
         steps=[('preprocessador',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('transformer_num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   

In [22]:
preds = pipeline_1.predict(X_valid_selec)

print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 18289.183561643833


In [0]:
preds_teste = pipeline_1.predict(X_teste_selec)

E pronto, agora basta salvarmos o resultado em um csv.

In [0]:
resultado = pd.DataFrame({'Id': X_teste_selec.index,
                       'SalePrice': preds_teste})
resultado.to_csv('resultado_pipeline.csv', index=False)